# **LOAN APPROVAL SYSTEM USING Z3 SOLVER**

Questo progetto implementa un sistema di valutazione di richieste di prestito utilizzando il constraint solver Z3 in Python.

L’obiettivo è modellare formalmente il processo decisionale di una banca tramite logica computazionale e vincoli matematici.

Il sistema determina se un richiedente può ottenere un prestito e, in caso positivo, calcola:

* tasso di interesse

* rata mensile

* interessi totali

## **STRUTTURA**

### **CLIENTE**

Classe ***Applicant***

Rappresenta un richiedente di prestito con i seguenti attributi:

- *name*: Nome del richiedente
- *age*: Età
- *work*: Tipo di impiego ('permanent', 'temporary', 'unemployed')
- *income*: Reddito mensile (€)
- *networth*: Patrimonio netto (€)
- *credit_score*: Punteggio di credito (0-1000)
- *requested*: Importo richiesto (€)
- *cosigner*: Presenza di un cofirmatario (True/False)
- *typeloan*: Tipo di prestito ('personal', 'car', 'house')
- *months*: Durata del prestito in mesi
- *blacklisted*: Se il cliente è nella blacklist della banca (True/False)

### **REGOLE DI APPROVAZIONE**

**Vincoli sull'età**

- Non approvato se età > 75 anni
- Non approvato se età ≤ 18 anni
- Non approvato se età ≤ 25 e assenza di cofirmatario
- L'età al termine del prestito non deve superare 85 anni: età + (mesi/12) ≤ 85

**Vincoli sul tipo di impiego**

- Disoccupato o temporaneo: Richiede cofirmatario
- Disoccupato: Il patrimonio netto deve essere almeno pari all'importo richiesto

**Vincoli sul tipo di prestito**

- Personale:
    - Importo massimo: €200.000
    - Durata: fino a 120 mesi
- Auto:
    - Importo massimo: €200.000
    - Età minima: > 25 anni
    - Durata: 12-120 mesi
- Mutuo casa: 
    - Importo minimo: €30.000
    - Durata: 60-360 mesi

**Vincoli sul patrimonio**

- Per prestiti > €100.000 (esclusi mutui):
    - Serve avere un lavoro a tempo indeterminato
    - Patrimonio ≥ 50% dell'importo richiesto

**Condizioni rischiose per ricevere un prestito**

- Età > 65 + mutuo casa + durata > 180 mesi
- Lavoro temporaneo + richiesta > €30.000 + assenza cofirmatario
- Credit score < 600 + reddito < €2.500 + assenza cofirmatario

**Sostenibilità della rata**

La rata mensile non deve superare:
- Mutuo casa: 50% del reddito mensile
- Altri prestiti: 20% del reddito mensile


### **CALCOLO DEL TASSO DI INTERESSE**

Il tasso viene calcolato sommando diverse componenti:

- Tasso base:
    - se età ≤ 35 : base_rate = 1 + (1000 - credit_score) × 0.007 + 0.2 × √(35 - età)
    
    - se età > 35: base_rate = 1 + (1000 - credit_score) × 0.007

- Tipo di prestito:
    - Mutuo casa: 0.0%
    - Altri prestiti: +4.5%

- Presenza cofirmatario:
    - Con cofirmatario e età ≤ 30: -0.5%
    - Con cofirmatario e età > 30: -0.3%
    - Senza cofirmatario: 0.0%

- Reddito:
    - ≥ €4.500 : 0.0%
    - €3.500 - €4.499 : +0.05%
    - €2.500 - €3.499 : +0.1%
    - €2.000 - €2.499 : +0.15%
    - < €2.000 : +0.2%

- DTI (debt-to-income):
    - Impiego permanente: 0.0%
    - Impiego temporaneo: richiesta / (reddito × mesi)
    - Impiego temporaneo: 0.2 + (richiesta / (reddito × mesi))

Tasso = base_rate + type_adj + cosigner_benefit + income_adj + dti_adj

### **CALCOLO DEL TASSO DI INTERESSE**

Rata mensile = (importo_richiesto / mesi) + (tasso/100 × importo_richiesto / 12)

### **ESEMPIO DI OUTPUT**

APPROVATO

Mario

Tasso: 2.12%

Reddito: €1500

Importo: €60000

Credit score: 850

Rata mensile (240 mesi): €355.83

Interessi totali: €25400.00

Totale dovuto: €85400.00